https://sonuc.ysk.gov.tr/module/ssps.jsf

In [7]:
import pandas as pd

In [2]:
PROVINCE = 'İL ADI'
df = pd.read_excel('results/turkey-june.xls').set_index(PROVINCE)
df.head()

,Seats,DYP,ANADOLU PARTİSİ,HAK-PAR,KP,MİLLET,HAP,MEP,TURK PARTİ,HKP,...,HDP,SAADET,CHP,AK PARTİ,DSP,YURT-P,DP,VATAN PARTİSİ,BTP,BAGIMSIZ TOPLAMI
İL ADI,,,,,,,,,,,,,,,,,,,,,
ADANA,14,1098,590,1265,310,509,0,643,2199,1432,...,176931,18906,354212,367912,1894,0,1534,3730,1730,5013
ADIYAMAN,5,338,75,428,52,103,239,199,1105,425,...,69390,6006,34795,178344,539,0,303,332,634,0
AFYONKARAHİSAR,5,524,208,0,61,391,0,292,1730,1111,...,4386,9264,71042,222400,1292,0,2186,878,721,0
AĞRI,4,0,104,334,34,57,0,0,448,331,...,185329,1311,2338,37609,217,0,100,60,625,1994
AKSARAY,3,136,74,362,33,144,75,147,0,477,...,3418,2689,13700,119873,528,46,297,233,441,0


In [75]:
parties = list(df.columns)
parties.remove('Seats')
if 'winner' in parties: parties.remove('winner')

In [4]:
def dHondt(name, row, parties):
    seats = int(row['Seats'])
    votes = [row[p] for p in parties]
    denominators = range(1,seats+1)
    coeffs = [[{ 'party' : parties[idx], 'coeff' : v/d} for d in denominators] for idx,v in enumerate(votes)]
    ranked_list = sorted([item for sublist in coeffs for item in sublist], key=lambda x:x['coeff'])
    winners = pd.DataFrame(ranked_list[-seats:])
    results = winners.groupby('party').count().rename(columns={'coeff':'seats'}).reset_index()
    results['province'] = name
    return results

In [8]:
dHondt('national', df.sum(), parties)

,party,seats,province
0,AK PARTİ,227,national
1,BAGIMSIZ TOPLAMI,6,national
2,BTP,1,national
3,CHP,140,national
4,DSP,1,national
5,HDP,72,national
6,MHP,91,national
7,SAADET,11,national
8,VATAN PARTİSİ,1,national


In [6]:
provinces = [dHondt(row[0],row[1],parties) for row in df.iterrows()]
pd.concat(provinces).groupby('party').sum()

,seats
party,
AK PARTİ,256
BAGIMSIZ TOPLAMI,1
CHP,133
HDP,78
MHP,82


In [16]:
df['winner'] = df.idxmax(axis=1)

In [57]:
df.groupby('winner').sum()['Seats']

winner
AK PARTİ    412
CHP          77
HDP          57
MHP           4
Name: Seats, dtype: int64

In [48]:
df_nov = pd.read_excel('results/turkey-november.xls').set_index(PROVINCE)
df_nov.head()

,Seats,MİLLET,VATAN PARTİSİ,CHP,HAK-PAR,SAADET,DSP,DP,BTP,MHP,HKP,LDP,HDP,BBP,AK PARTİ,KP,DYP,BAGIMSIZ TOPLAMI
İL ADI,,,,,,,,,,,,,,,,,,
ADANA,14,481,2838,373629,2578,7998,773,1447,1088,246053,2055,867,146909,4864,462199,602,0,0
ADIYAMAN,5,95,333,35948,693,2061,109,217,175,9467,428,220,45294,1824,222121,542,107,0
AFYONKARAHİSAR,5,319,808,69995,0,3411,305,1701,481,73375,956,134,2343,3052,273718,640,311,0
AĞRI,4,49,83,2783,491,672,127,136,174,3274,403,814,145535,1202,57767,123,0,0
AKSARAY,3,110,194,15561,491,859,112,273,311,38248,451,0,1627,1264,151403,367,73,0


In [80]:
parties_nov = list(df_nov.columns)
parties_nov.remove('Seats')
if 'winner' in parties_nov: parties_nov.remove('winner')
dHondt('national', df_nov.sum(), parties_nov)

,party,seats,province
0,AK PARTİ,276,national
1,BBP,2,national
2,CHP,142,national
3,HAK-PAR,1,national
4,HDP,58,national
5,MHP,67,national
6,SAADET,3,national
7,VATAN PARTİSİ,1,national


In [56]:
df_nov['winner'] = df_nov[parties_nov].idxmax(axis=1)
df_nov.groupby('winner').sum()['Seats']

winner
AK PARTİ    446
CHP          51
HDP          53
Name: Seats, dtype: int64

In [92]:
vote_tallies = df[parties].sum().sort_values()
dq_parties = vote_tallies[:-4]
dq_parties.sum()

2163533

In [93]:
dq_parties.sum()/vote_tallies.sum()

0.047981308443962735

In [95]:
nov_tallies = df_nov[parties_nov].sum().sort_values()
dq_parties_nov = nov_tallies[:-4]
dq_parties_nov.sum()

1177583